In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
import io
from PIL import Image
from time import sleep
import time
import random
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import os
import pandas as pd
import numpy as np


In [ ]:
#Crawl basic phone data
# Declare browser
s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

# Khởi tạo trình duyệt Chrome với tùy chọn toàn màn hình
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=s, options=chrome_options)

# Open URL
# driver.get("https://www.thegioididong.com/dtdd-apple-iphone")
driver.get("https://www.thegioididong.com/dtdd#c=42&m=80,17201,2,1971,2235,2236&o=17&pi=7")
# driver.get("https://www.thegioididong.com/dtdd-samsung")
# driver.get("https://www.thegioididong.com/dtdd-oppo")
sleep(random.randint(5,10))

# Scroll down
for i in range(5):
    scroll_script = "window.scrollBy(0, window.innerHeight);"
    driver.execute_script(scroll_script)
    time.sleep(1) 

# Scroll up
for i in range(20):
    scroll_script = "window.scrollBy(0, -window.innerHeight);"
    driver.execute_script(scroll_script)
    time.sleep(1)
    
# elems2 = driver.find_elements(By.CLASS_NAME , "jBwCF ")
# links = [elem.get_attribute('src') for elem in elems2 if elem.get_attribute('type') == 'product' and elem.get_attribute('src').count("data:image") <=0]

# ================================ GET id
elems = driver.find_elements(By.CSS_SELECTOR, '.item.ajaxed.__cate_42 a')
id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]

# # ================================ GET title
elems = driver.find_elements(By.CSS_SELECTOR, '.item.ajaxed.__cate_42 a')
title = [elem.get_attribute('data-name') for elem in elems if elem.get_attribute('data-name') is not None]

# ================================ GET img_thumb
# elems = driver.find_elements(By.CLASS_NAME, 'thumb')
elems = driver.find_elements(By.CSS_SELECTOR, '.item-img.item-img_42 img')
link_img_thumb = [elem.get_attribute('src') for elem in elems if elem.get_attribute('src') is not None and "Products" in elem.get_attribute('src')]

# ================================ GET href
elems = driver.find_elements(By.CSS_SELECTOR, '.item.ajaxed.__cate_42 a')
href = [elem.get_attribute('href') for i, elem in enumerate(elems) if i % 3 == 0]

# ================================ GET price
elems = driver.find_elements(By.CSS_SELECTOR, '.item.ajaxed.__cate_42 a')
price = [elem.get_attribute('data-price') for elem in elems if elem.get_attribute('data-price') is not None]

# Clean price
price = [int(float(number)) for number in price]

# ================================ GET price pre discount
discount_list, discount_idx, discount_percent_list = [], [], []
discount_list, discount_idx, discount_percent_list = [], [], []
for i in range(1, len(title)+1):
    try:

        try:
            discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[5]/span".format(i))
        except NoSuchElementException:
            discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[4]/span".format(i))

            
        try:
            discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[4]/p".format(i))
        except NoSuchElementException:
            discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[5]/p".format(i))

        discount_list.append(discount.text)

        

        discount_percent_list.append(discount_percent.text)
        print(i)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))


# ================================ GET brand
elems = driver.find_elements(By.CSS_SELECTOR, '.item.ajaxed.__cate_42 a')
brand = [elem.get_attribute('data-brand') for elem in elems if elem.get_attribute('data-brand') is not None]


# Close browser
driver.close() 

In [3]:
#Clean discount_list
for i, number in enumerate(discount_list):
    if discount_list[i] and discount_list[i] != '':
        if '.' in discount_list[i]:
            discount_list[i] = int(discount_list[i].replace('.', '').replace('₫', ''))

In [5]:
#Merge & create excel
df1 = pd.DataFrame(list(zip(id_product, title, brand ,href ,link_img_thumb , price)), columns = ['id', 'title' ,'brand', 'href', 'link_img_thumb', 'price'])
# Đặt tên tệp Excel và sheet
excel_file = 'phoneupdate.xlsx'
sheet_name = 'Sheet1'

df1['index_']= np.arange(1, len(df1) + 1)

df2 = pd.DataFrame(list(zip(discount_list ,discount_percent_list, discount_idx)), columns = ['discount', 'discount_percent','discount_idx'])
df3 = df1.merge(df2, how='left', left_on='index_', right_on='discount_idx')

# Lưu DataFrame vào tệp Excel
df3.to_excel(excel_file, sheet_name=sheet_name, index=False, engine='openpyxl')

In [ ]:
len(link_img_thumb)

In [ ]:
df = pd.read_excel('phoneupdate.xlsx')

link_img_thumb = df['link_img_thumb'].to_list()
id_product = df['id'].to_list()

print(link_img_thumb)

In [ ]:
def download_image(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{id_product[i]}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


urls = link_img_thumb

dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
for i, url in enumerate(urls):
	download_image(dowloadPath, url, i)

In [16]:
#Crawl parameters data
screen =[]
operating_system =[]
back_camera =[]
front_camera =[]
chip = []
ram = []
rom = []
sim = []
battery_charger = [] 
list_parameters = []
for href_items in href:
# for i in range(1):

    # Declare browser
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    # Browser full window
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    #Open browser 
    driver.get(href_items)
    sleep(random.randint(1,2))

    # Scroll down
    for i in range(5):
        scroll_script = "window.scrollBy(0, window.innerHeight);"
        driver.execute_script(scroll_script)
        time.sleep(1) 

    elems = driver.find_elements(By.CSS_SELECTOR, '.parameter')
    # id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]
    parameters = [elem.text for elem in elems]

    list_parameters.append(parameters[0])



    #Close browser
    driver.close()

In [ ]:
list_parameters

In [25]:
#Clean parameters data
for data in list_parameters:
    data = data.replace("Xem thêm cấu hình chi tiết", "")
    # Tách dữ liệu thành danh sách các dòng
    data = data.replace('\n', ' ')
    # Tìm vị trí của các thông số trong dữ liệu
    if "Hướng" in data:
        index = data.find("Hướng")
        data = data[0:index]
    start_index = {
        "Màn hình": data.find("Màn hình: "),
        "Hệ điều hành": data.find("Hệ điều hành: "),
        "Camera sau": data.find("Camera sau: "),
        "Camera trước": data.find("Camera trước: "),
        "Chip": data.find("Chip: "),
        "RAM": data.find("RAM: "),
        "Dung lượng lưu trữ": data.find("Dung lượng lưu trữ: "),
        "SIM": data.find("SIM: "),
        "Pin, Sạc": data.find("Pin"),
    }

    # Tìm vị trí của các thông số kết thúc
    end_index = {
        "Màn hình": data.find(" Hệ điều hành:"),
        "Hệ điều hành": data.find("Camera sau:"),
        "Camera sau": data.find(" Camera trước:"),
        "Camera trước": data.find(" Chip:"),
        "Chip": data.find(" RAM:"),
        "RAM": data.find(" Dung lượng lưu trữ:"),
        "Dung lượng lưu trữ": data.find(" SIM:"),
        "SIM": data.find("Pin"),
        "Pin, Sạc": len(data) ,
    }

    # Khởi tạo một từ điển trống
    phone_info = {}

    # Duyệt qua các thông số để lấy giá trị
    for key in start_index.keys():
        if start_index[key] != -1 and end_index[key] != -1:
            value = data[start_index[key] + len(key + ": "):end_index[key]]
            phone_info[key] = value

    if 'Màn hình' in phone_info:
        screen.append(phone_info['Màn hình'])
    else:
        screen.append(None)
    if 'Hệ điều hành' in phone_info:
        operating_system.append(phone_info['Hệ điều hành'])
    else:
        operating_system.append(None)
    if 'Camera sau' in phone_info:
        back_camera.append(phone_info['Camera sau'])
    else:
        back_camera.append(None) 
    if 'Camera trước' in phone_info:
        front_camera.append(phone_info['Camera trước'])
    else:
        front_camera.append(None) 
    if 'Chip' in phone_info:
        chip.append(phone_info['Chip'])
    else:
        chip.append(None) 
    if 'RAM' in phone_info:
        ram.append(phone_info['RAM'])
    else:
        ram.append(None) 
    if 'Dung lượng lưu trữ' in phone_info:
        rom.append(phone_info['Dung lượng lưu trữ'])
    else:
        rom.append(None)
    if 'SIM' in phone_info:
        sim.append(phone_info['SIM'])
    else:
        sim.append(None)
    if 'Pin, Sạc' in phone_info:
        battery_charger.append(phone_info['Pin, Sạc'])
    else:
        battery_charger.append(None)


In [26]:
new_df2 = pd.DataFrame(list(zip(screen, operating_system, back_camera ,front_camera ,chip, ram, rom, sim, battery_charger)), columns = ['screen', 'operating_system', 'back_camera', 'front_camera', 'chip', 'ram', 'rom', 'sim', 'battery_charger'])
new_df2.to_excel('phone_info_update.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [38]:
len(screen)

144

In [24]:
def download_image_details(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{index}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


# urls = link_img_thumb

# parent_folder = r'D:\CodeUIT\Data\imgs'
# folder_name = 'abc'

# folder_path = os.path.join(parent_folder, folder_name)

# os.makedirs(folder_path, exist_ok=True)

# dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
# for i, url in enumerate(urls):
# 	download_image(dowloadPath, url, i)

In [111]:
df = pd.read_excel('phoneupdate.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()

In [117]:
href_test

['https://www.thegioididong.com/dtdd/iphone-15-pro-max-512gb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-max',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-1tb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-512gb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-256gb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro']

In [115]:
href_test = []
for i, data in enumerate(href):
    href_test.append(data)
    if i==5: break

In [121]:
id_failed = []
id_success = []
for i,href_items in enumerate(href):
# for i in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    # driver.get("https://www.thegioididong.com/dtdd-apple-iphone")
    # driver.get("https://www.thegioididong.com/dtdd/samsung-galaxy-s23-fe")
    # driver.get("https://www.thegioididong.com/dtdd-samsung")
    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    for scroll_i in range(2):
        scroll_script = "window.scrollBy(0, window.innerHeight);"
        driver.execute_script(scroll_script)
        time.sleep(1) 

    # # Scroll up
    # for i in range(5):
    #     scroll_script = "window.scrollBy(0, -window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1)


    # elems = driver.find_elements(By.CSS_SELECTOR, '.parameter-item')
    # parameters_details = [elem.text for elem in elems]

    elems = driver.find_elements(By.CSS_SELECTOR, '.owl-stage .owl-item.active a img')
    pics_act = [elem.get_attribute('src') for elem in elems ]

    elems = driver.find_elements(By.CSS_SELECTOR, '.owl-stage .owl-item a img' )
    pics = [elem.get_attribute('data-src') for elem in elems  ]

    pics_act_clean = []
    pics_clean = []

    for data in pics_act:
        if data and isinstance(data, str) and 'Slider' in data:
            pics_act_clean.append(data)
    
    for data in pics:
        if data and isinstance(data, str) and 'Slider' in data:
            pics_clean.append(data)

    # print(pics_act_clean)
    # print(pics_clean)

    pics_act = pics_act_clean
    pics = pics_clean

    if pics_act and pics:
        pics.insert(0, pics_act[0])
    
        urls = pics

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_product[i])

        folder_path = os.path.join(parent_folder, folder_name)

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path

        for z, url in enumerate(urls):
            download_image_details(dowloadPath, url, z)

        id_success.append(id_product[i])
        print('------Successful ', i)
    else:
        id_failed.append(id_product[i])


   
    driver.close()

Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
------Successful  0
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
------Successful  1
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  2
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  3
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  4
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  5
FAILED - cannot write mode RGBA as JPEG
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
FAILED - cannot write mode RGBA as JPEG
Success  8
------Successfu

In [123]:
id_failed

[313194,
 302953,
 309835,
 314975,
 306994,
 317163,
 317164,
 317981,
 312966,
 275953,
 309864,
 314203,
 313202,
 313199,
 316771,
 317221]

In [ ]:
i=0
for data in pics_clean:
    print(data, '\n')
    i+=1
    

print(i)

In [74]:
id_success

[]

In [45]:
i=0
for data in pics_act:
    if data is not None and '305659' in data and 'thumb' not in data:
        print(data, '\n')
        i+=1
print(i)
# for data in pics_act:
#     if data is not None :
#         print(data, '\n')
print(pics_act)

0
[]


In [43]:
df = pd.read_excel('phoneupdate.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()
print(id_product)

[305659, 305658, 303833, 303832, 303831, 299033, 303825, 303823, 303891, 303812, 303716, 281570, 307245, 251192, 309814, 305695, 306980, 306979, 285082, 283828, 309816, 309846, 299734, 303163, 234621, 306785, 305887, 301603, 292672, 311355, 310805, 309746, 289705, 301608, 301796, 289702, 317163, 317164, 289694, 249948, 301798, 309745, 289700, 285031, 290829, 289691, 299250, 247508, 289710, 250625, 264060, 289663, 245545, 250258, 250622, 240259, 299034, 235838, 223602, 228736, 285696, 260546, 213031, 309821, 258047, 210644, 267212, 315401, 153856, 304222, 314358, 267211, 282389, 309722, 250103, 303585, 301642, 302531, 278886, 303179, 307891, 297026, 247361, 298377, 301641, 309834, 271717, 306774, 303583, 299248, 283148, 304162, 309847, 310647, 307203, 305886, 311120, 273335, 274018, 311354, 304090, 292780, 303298, 285224, 275435, 286697, 271721, 317530, 292770, 291696, 313154, 304182, 315651, 317974, 313320, 303579, 313153, 307556, 288401, 291697, 283819, 291625, 288404, 304276, 309723,

In [93]:
color_item = []
for index, href_items in enumerate(href):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_product[index])

    elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    list_color = [elem.text for elem in elems if elem.text is not None]

    len_color = len(elems)
    len_loop = len_color+2
    
    # i = 2 
    # while i < len_loop:
    #     elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
    #     elem_find.click()

    #     elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active')
    #     check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

    #     if check[0] != 'color-images-gallery':
    #         print(check)
    #         len_loop += 1
    #         i+=1
    #         continue
    #     else:
    #         i += 1

        # wait = WebDriverWait(driver, 10)
        # element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')))
        # sleep(2)

        # elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        # color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]

        # elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        # thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        # elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        # href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]
        

        # for item in href_color:
        #     if item is None or str(id_product[index]) not in item:
        #         href_color.remove(item)

        # urls = href_color

        # parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        # folder_name = str(id_product[index])
        # folder_path = os.path.join(parent_folder, folder_name, color[0])

        # os.makedirs(folder_path, exist_ok=True)

        # dowloadPath = folder_path
        
        # download_image_details(dowloadPath, thumb_color[0], 'thumb')

        # for z, url in enumerate(urls):
        #     url = 'https:' + url
        #     download_image_details(dowloadPath, url, z)
    color_item.append(', '.join(list_color))
        

    driver.close()

305659
305658
303833
303832
303831
299033
303825
303823
303891
303812
303716
281570
307245
251192
309814
305695
306980
306979
285082
283828
309816
309846
299734
303163
234621
306785
305887
301603
292672
311355
310805
309746
289705
301608
301796
289702
317163
317164
289694
249948
301798
309745
289700
285031
290829
289691
299250
247508
289710
250625
264060
289663
245545
250258
250622
240259
299034
235838
223602
228736
285696
260546
213031
309821
258047
210644
267212
315401
153856
304222
314358
267211
282389
309722
250103
303585
301642
302531
278886
303179
307891
297026
247361
298377
301641
309834
271717
306774
303583
299248
283148
304162
309847
310647
307203
305886
311120
273335
274018
311354
304090
292780
303298
285224
275435
286697
271721
317530
292770
291696
313154
304182
315651
317974
313320
303579
313153
307556
288401
291697
283819
291625
288404
304276
309723
275434
303575
268880
302655
302654


In [107]:
for index, href_items in enumerate(href_failed):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_failed[index])

    elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    list_color = [elem.text for elem in elems if elem.text is not None]

    len_color = 1
    if len(elems) != 0:
        len_color = len(elems) 
    
    len_loop = len_color+2
    
    i = 1 
    while i < len_loop:
        elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
        elem_find.click()

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active')
        check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

        if len(check) >0 :
            if check[0] != 'color-images-gallery' and check[0]:
                print(check)
                len_loop += 1
                i+=1
                continue
            else:
                i += 1
        else: break

        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')))
        sleep(2)

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]
        

        for item in href_color:
            if item is None or str(id_failed[index]) not in item:
                href_color.remove(item)

        urls = href_color

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_failed[index])
        folder_path = os.path.join(parent_folder, folder_name, color[0])

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path
        
        download_image_details(dowloadPath, thumb_color[0], 'thumb')

        for z, url in enumerate(urls):
            url = 'https:' + url
            download_image_details(dowloadPath, url, z)
    # color_item.append(', '.join(list_color))
    color_item[index_failed[index]] = color[0]
        

    driver.close()

305695
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
317163
Success  thumb
Success  0
Success  1
Success  2
Success  3
FAILED - cannot write mode P as JPEG
317164
Success  thumb
Success  0
Success  1
Success  2
Success  3
FAILED - cannot write mode P as JPEG
289694
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
250622
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
285696
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
Success  14
Success  15
309821
['featured-images-gallery']
Succes

In [105]:
index_failed = []
for it in id_failed:
    for x, item in enumerate(id_product):
        if it == item:
            index_failed.append(x)

In [109]:
new_df2 = pd.DataFrame(list(zip(color_item)), columns = ['color_item'])
new_df2.to_excel('color_item.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [108]:
color_item

['Titan xanh, Titan đen, Titan tự nhiên, Titan trắng',
 'Titan trắng, Titan đen',
 'Titan tự nhiên, Titan đen, Titan xanh, Titan trắng',
 'Titan đen, Titan tự nhiên, Titan trắng, Titan xanh',
 'Titan đen, Titan xanh, Titan tự nhiên, Titan trắng',
 'Titan tự nhiên, Titan xanh, Titan trắng, Titan đen',
 'Xanh lá nhạt, Đen, Xanh dương nhạt, Hồng nhạt, Vàng nhạt',
 'Đen, Xanh dương nhạt, Vàng nhạt, Hồng nhạt, Xanh lá nhạt',
 'Xanh dương nhạt, Đen, Hồng nhạt, Vàng nhạt, Xanh lá nhạt',
 'Xanh dương nhạt, Xanh lá nhạt, Hồng nhạt, Đen, Vàng nhạt',
 'Hồng nhạt, Vàng nhạt, Xanh lá nhạt, Xanh dương nhạt, Đen',
 'Hồng nhạt, Xanh dương nhạt, Đen, Vàng nhạt, Xanh lá nhạt',
 'Xanh Dương, Bạc, Đen',
 'Tím, Đen, Vàng',
 'Xanh Dương, Xanh lá, Đen',
 'Xanh Dương',
 'Tím, Xám',
 'Xám, Tím',
 'Đen, Xanh lá, Vàng',
 'Đen, Vàng',
 'Xanh lá, Xanh Dương, Đen',
 'Vàng, Xanh lá, Xám, Xanh Dương',
 'Xanh dương nhạt, Xám, Xanh Dương',
 'Xám, Xanh Dương, Xanh lá',
 'Xám, Xanh Dương, Tím',
 'Vàng, Đen',
 'Xanh Dương